In [1]:
print("hello world")

hello world


In [12]:
import pandas as pd
data_path = '../data/skylab_instagram_datathon_dataset.csv'

data = pd.read_csv(data_path, delimiter=';')  # Specify the delimiter (e.g., comma)

In [8]:
print(len(data))

704313


In [1]:
print(data[:5])

NameError: name 'head' is not defined